2. For SMS Spam Collection Dataset build a Naive Bayes Classifer. The files
contain one message per line. Each line is composed by two columns: v1
contains the label (ham or spam) and v2 contains the raw text. Divide data
to test and train. Calculate the error on test data

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [37]:
df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")[['v1', 'v2']].rename({'v1':'class', 'v2':'message'}, axis=1)
df

,class,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


Препоцессинг данных

In [38]:
df['class_num'] = df['class'].apply(lambda x: int(x == 'spam')) # 1 - spam, 0 - not spam
bad_sample = df.query('class_num == 1')['message'].to_list()
good_sample = df.query('class_num == 0')['message'].to_list()
df

,class,message,class_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will Ì_ b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [39]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
import re


stop = stopwords.words('english')

def clean_text(text):
  # Приводим к нижнему регистру
  text = text.lower()
  # Удаляем ссылки знаки препинания
  text = re.sub(r"/(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|www\S+|http\S+/g", "", text)
  # Удаляем стоп-слова
  text = " ".join([word for word in text.split() if word not in (stop)])
  return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vital\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
print(bad_sample[5])
print(clean_text(bad_sample[5]))

URGENT! You have won a 1 week FREE membership in our å£100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18
urgent 1 week free membership 100000 prize jackpot txt word claim 81010 tc lccltd pobox 4403ldnw1a7rw18


In [26]:
print(good_sample[5])
print(clean_text(good_sample[5]))

As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
per request melle melle oru minnaminunginte nurungu vettam set callertune callers press 9 copy friends callertune


In [27]:
df['message'] = df['message'].apply(clean_text)

<h3> Деление на тестовую и обучающую выборку

Разделим выборку на обучающую и тестовую так, чтобы пропорции классов сохранились

In [40]:
df.class_num.value_counts(normalize=True)*100

class_num
0    86.593683
1    13.406317
Name: proportion, dtype: float64

In [30]:
X_train, X_test, y_train, y_test = train_test_split(df['message'].values, df['class_num'].values, test_size=0.20, stratify=df['class_num'], random_state=42)

In [31]:
print("TRAIN Class 0: {:.3f}%".format((y_train == 0).sum()/(y_train.shape[0])*100))
print("TRAIN Class 1: {:.3f}%".format(y_train.sum()/(y_train.shape[0])*100))
print("TEST Class 0: {:.3f}%".format((y_test == 0).sum()/(y_test.shape[0])*100))
print("TEST Class 1: {:.3f}%".format(y_test.sum()/(y_test.shape[0])*100))

TRAIN Class 0: 86.583%
TRAIN Class 1: 13.417%
TEST Class 0: 86.637%
TEST Class 1: 13.363%


In [32]:
def make_dataset(goods: list, bads: list):
    dataset = []
    for text in goods:
        words = text.split(sep=' ')
        temp = []
        for word in words:
          if len(word) > 3:
            temp += [word]
        dataset += [[temp, 0]]
        
    for text in bads:
        words = text.split(sep=' ')
        temp = []
        for word in words:
          if len(word) > 3:
            temp += [word]
        dataset += [[temp, 1]]
    return dataset

In [14]:
good_train = X_train[y_train == 0]
bad_train = X_train[y_train == 1]
good_test = X_test[y_test == 0]
bad_test = X_test[y_test == 1]

In [41]:
train_dataset = make_dataset(good_train, bad_train)
test_dataset = make_dataset(good_test, bad_test)

[[['going', 'nothing', 'greatbye'], 0],
 [['wont', 'wats', 'guys'], 0],
 [['ksry', 'sivatats', 'askd'], 0],
 [['stand',
   'away',
   'doesnt',
   'heart',
   'ache',
   'without',
   'dont',
   'wonder',
   'dont',
   'crave'],
  0],
 [['finished', 'work', 'something'], 0],
 [['hhahhaahahah', 'rofl', 'leonardo', 'room', 'something'], 0],
 [[], 0],
 [['hiya',
   'sorry',
   'didnt',
   'signal',
   'havent',
   'seen',
   'heard',
   'neither',
   'unusual',
   'case',
   'sort',
   'hugs',
   'snogs'],
  0],
 [['outside', 'already'], 0],
 [['schedule', 'next', 'week', 'town', 'weekend'], 0],
 [['goodmorning', 'today', 'late'], 0],
 [['dizzamn', 'aight', 'suitemates', 'back'], 0],
 [['must',
   'around',
   'wait',
   'summer',
   'days',
   'celebrate',
   'magical',
   'sight',
   'worlds',
   'dressed',
   'white',
   'oooooh',
   'snow'],
  0],
 [['call', 'happen'], 0],
 [['awake', 'snow'], 0],
 [['nope', 'since', 'travelled', 'forgotten'], 0],
 [['friendship',
   'game',
   'play'

Своя реализация

In [16]:
import numpy as np
import pickle

class NaiveBayesClf:
    
    def __init__(self, alpha=0.01):
        self.classes = {}
        self.freq = {}
        self.total_in_class = {}
        self.total = set()
        self.alpha = alpha
    
    def fit(self, dataset):
        self.dataset = dataset
        for features, label in self.dataset:
            if label not in self.classes:
                self.classes[label] = 0
                self.total_in_class[label] = 0
            self.classes[label] += 1
            for feature in features:
                if (feature, label) not in self.freq:
                    self.freq[(feature, label)] = 0
                self.freq[(feature, label)] += 1
                self.total_in_class[label] += 1
                self.total.add(feature)
    
        for feature, label in self.freq:
            self.freq[(feature, label)] = (self.alpha + self.freq[(feature, label)])/(self.alpha*len(self.total) + self.total_in_class[label])
        for cls in self.classes:
            self.classes[cls] /= len(self.dataset)
        return self
        

    def predict(self, features):
        return max(self.classes.keys(), 
                   key = lambda cls: np.log10(self.classes[cls]) + \
                   sum(np.log10(self.freq.get((feature, cls), self.alpha/(self.alpha*len(self.total)+self.total_in_class[cls])))\
                                                                             for feature in features))
    
    def save(self,path:str):
        dump = {
            'classes':self.classes,
            'freq':self.freq,
            'total_in_class':self.total_in_class,
            'total':self.total,
            'alpha':self.alpha
        }
        with open(path, 'wb') as f:
            pickle.dump(dump, f)
            
    def load(self, path:str):

        with open(path, 'rb') as f:
            dump = pickle.load(f)
        self.classes = dump['classes']
        self.freq = dump['freq']
        self.total_in_class = dump['total_in_class']
        self.total = dump['total']
        self.alpha = dump['alpha']
        return self

In [17]:
model = NaiveBayesClf(alpha=1).fit(train_dataset)

In [18]:
train_preds = []
for item in X_train:
  train_preds += [model.predict(item.split())]
train_preds = np.array(train_preds)

In [19]:
test_preds = []
for item in X_test:
  test_preds += [model.predict(item.split())]
test_preds = np.array(test_preds)

In [20]:
print("Точность на обучающей выборке: " + str(accuracy_score(y_train, train_preds)))
print("Точность на тестовой выборке: " + str(accuracy_score(y_test, test_preds)))

NameError: name 'accuracy_score' is not defined

In [ ]:
print(classification_report(y_train, train_preds))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      3859
           1       0.79      0.97      0.87       598

    accuracy                           0.96      4457
   macro avg       0.89      0.97      0.93      4457
weighted avg       0.97      0.96      0.96      4457



In [ ]:
print(classification_report(y_test, test_preds))

              precision    recall  f1-score   support

           0       0.99      0.91      0.95       966
           1       0.62      0.97      0.75       149

    accuracy                           0.92      1115
   macro avg       0.81      0.94      0.85      1115
weighted avg       0.94      0.92      0.92      1115



Проверим на реализции из sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

vect = CountVectorizer()              
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_test_dtm = vect.transform(X_test) 

nb = MultinomialNB() 
nb.fit(X_train_dtm, y_train) 

y_pred_train = nb.predict(X_train_dtm)
print("Точность на обучающей выборке: " + str(accuracy_score(y_train, y_pred_train)))

y_pred_test = nb.predict(X_test_dtm)
print("Точность на тестовой выборке: " + str(accuracy_score(y_test, y_pred_test)))

Точность на обучающей выборке: 0.9930446488669509
Точность на тестовой выборке: 0.9802690582959641


In [ ]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3859
           1       0.97      0.97      0.97       598

    accuracy                           0.99      4457
   macro avg       0.98      0.99      0.99      4457
weighted avg       0.99      0.99      0.99      4457



In [ ]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       0.97      0.88      0.92       149

    accuracy                           0.98      1115
   macro avg       0.98      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115

